In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import datetime

# Function Definitions
def get_historical_data(ticker, period):
    data = yf.download(ticker, period=period)
    data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
    data['MACD'] = ta.trend.macd_diff(data['Close'])
    data['BollingerHigh'] = ta.volatility.bollinger_hband(data['Close'])
    data['BollingerLow'] = ta.volatility.bollinger_lband(data['Close'])
    data.dropna(inplace=True)
    return data

def train_model(data):
    X = data[['RSI', 'MACD', 'BollingerHigh', 'BollingerLow']]
    y = data['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f'Mean Squared Error: {mse}')
    return model, scaler

def predict_and_visualize(model, scaler, data, prediction_period, ticker):
    last_date = data.index[-1]
    future_dates = [last_date + datetime.timedelta(days=i) for i in range(1, prediction_period + 1)]
    X_future = scaler.transform(data[['RSI', 'MACD', 'BollingerHigh', 'BollingerLow']].iloc[-1].values.reshape(1, -1))
    predictions = []
    for i in range(prediction_period):
        prediction = model.predict(X_future)
        predictions.append(prediction[0])
        X_future = np.roll(X_future, -1)
        X_future[0][-1] = prediction
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data['Close'], label="Historical Prices")
    plt.plot(future_dates, predictions, label="Predicted Prices", linestyle='--')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f"Price Prediction for {ticker}")
    plt.legend()
    plt.show()

# Example usage
ticker = 'AAPL'
period = '1y'
prediction_period = '1M'
data = get_historical_data(ticker, period)
model, scaler = train_model(data)
predict_and_visualize(model, scaler, data, 30, ticker)
